# **Conceptos Fundamentales de azure-ai-ml SDK v2**

## **1. `azure.ai.ml`**
 - SDK (Software Development Kit) moderno y oficial de Azure Machine Learning, diseñado para interactuar con recursos de Azure ML mediante programación. 
 - Proporciona una interfaz simplificada para gestionar experimentos, datos, computación, modelos y pipelines, facilitando la creación y operacionalización de soluciones de machine learning en la nube.

 https://learn.microsoft.com/en-us/python/api/overview/azure/ai-ml-readme?view=azure-python

## **2. Instalación**
Para comenzar a usar `azure.ai.ml`, instala el paquete utilizando `pip`:

In [1]:
# ! pip install azure-ai-ml

También es recomendable instalar las dependencias adicionales necesarias, como `azure-identity` para autenticación:

In [ ]:
# ! pip install azure-identity

## **3. Configuración Inicial**

### Crear un Workspace
Un **Workspace** es el recurso central que organiza todos los recursos de Azure ML. Puedes crearlo desde Azure Portal o mediante la CLI de Azure:

```bash
az ml workspace create --name <workspace_name> --resource-group <resource_group> --location <location>
```

### Conectarse al Workspace
Usa el SDK para conectarte al Workspace mediante autenticación (`DefaultAzureCredential`) y un archivo de configuración (`config.json`):

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Conexión usando config.json
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# O especificando manualmente los parámetros
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="<subscription_id>",
    resource_group_name="<resource_group>",
    workspace_name="<workspace_name>"
)

print(f"Conectado al Workspace: {ml_client.workspace_name}")


## **4. Experimentos**
Los **experimentos** son colecciones de ejecuciones (runs) que agrupan pruebas o entrenamientos de modelos. Puedes registrar métricas y artefactos durante cada ejecución.

### Ejemplo: Ejecutar un Experimento


In [ ]:
from azure.ai.ml import command, MLClient
from azure.ai.ml.entities import Environment

# Definir un comando para el experimento
job = command(
    code="./src",  # Carpeta con el código
    command="python train.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="my-cluster",
    display_name="mi-experimento"
)

# Enviar el experimento
returned_job = ml_client.jobs.create_or_update(job)
print(f"Experimento enviado: {returned_job.name}")


## **5. Datos**
Los **datos** se gestionan mediante conjuntos de datos (`Data`) registrados en el Workspace. Pueden ser tabulares, archivos u otros formatos.

### Ejemplo: Registrar un Dataset


In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Crear un dataset
my_data = Data(
    path="azureml://datastores/my_datastore/paths/dataset.csv",
    type=AssetTypes.URI_FILE,
    description="Dataset de ejemplo",
    name="mi-dataset"
)

# Registrar el dataset
ml_client.data.create_or_update(my_data)
print(f"Dataset registrado: {my_data.name}")

## **6. Computación**
La **computación** define los recursos utilizados para ejecutar trabajos, como clústeres de CPU/GPU o Kubernetes.

### Ejemplo: Crear un Clúster de Computación


In [ ]:
from azure.ai.ml.entities import AmlCompute

# Crear un clúster de computación
compute_cluster = AmlCompute(
    name="mi-cluster",
    type="amlcompute",
    size="Standard_D2_v2",
    min_instances=0,
    max_instances=4
)

# Registrar el clúster
ml_client.compute.begin_create_or_update(compute_cluster)
print(f"Clúster creado: {compute_cluster.name}")

## **7. Entrenamiento de Modelos**
El entrenamiento de modelos se realiza mediante trabajos (`jobs`) que ejecutan scripts de entrenamiento en recursos computacionales.

### Ejemplo: Entrenar un Modelo

In [ ]:
from azure.ai.ml import command

# Definir el trabajo de entrenamiento
train_job = command(
    code="./src",
    command="python train.py --learning_rate 0.01 --epochs 10",
    environment="AzureML-tensorflow-2.7-ubuntu18.04-py37-cpu@latest",
    compute="mi-cluster",
    display_name="entrenamiento-modelo"
)

# Enviar el trabajo
returned_job = ml_client.jobs.create_or_update(train_job)
print(f"Entrenamiento enviado: {returned_job.name}")

## **8. Registro de Modelos**
Los **modelos** entrenados se registran en el Workspace para su versionado y reutilización.

### Ejemplo: Registrar un Modelo

In [ ]:
from azure.ai.ml.entities import Model

# Registrar un modelo
registered_model = Model(
    path="azureml://datastores/my_datastore/paths/model.pkl",
    name="mi-modelo",
    description="Modelo entrenado para clasificación"
)

ml_client.models.create_or_update(registered_model)
print(f"Modelo registrado: {registered_model.name}")

## **9. Despliegue de Modelos**
Los modelos registrados se pueden desplegar como servicios web RESTful utilizando **endpoints**.

### Ejemplo: Desplegar un Modelo


In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# Crear un endpoint
endpoint = ManagedOnlineEndpoint(
    name="mi-endpoint",
    description="Endpoint para predicciones en tiempo real",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint)

# Crear una implementación
deployment = ManagedOnlineDeployment(
    name="mi-deployment",
    endpoint_name="mi-endpoint",
    model="mi-modelo:1",
    instance_type="Standard_DS2_v2",
    instance_count=1
)

ml_client.online_deployments.begin_create_or_update(deployment)
print(f"Modelo desplegado en: {endpoint.name}")

## **10. Referencias**
- [Documentación Oficial de `azure.ai.ml`](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-ml-readme?view=azure-python)
- [Azure Machine Learning Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/)
- [GitHub Repository for `azure-ai-ml`](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ml/azure-ai-ml)
